In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import json
import re
import pandas as pd

project = "import agrifootprint"
bd.projects.set_current(project)

In [2]:
# Biosphere
bi.bw2setup()
bio = bd.Database("biosphere3")

Biosphere database already present!!! No setup is needed


In [36]:
def add_location(db):
    location_pattern = r"\{(.*?)\}"
    for act in af.data:
        for exc in act.get('exchanges'):
            if exc.get('location') is None:
                match = re.findall(pattern=location_pattern, string=exc['name'])
                try:
                    exc['location'] = match[0]
                    if exc.get('type') == "production":
                        act['location'] = match[0]
                except:
                    pass
    return db

In [3]:
# Technosphere, add location to electricity and heat
def add_location_energy_acts(db):
    location_pattern = r"\{(.*?)\}"
    for act in af.data:
        for exc in act.get('exchanges'):
            if (
                ("Electricity, low voltage" in exc.get('name')
                 or "Heat" in exc.get('name'))
                and "ecoinvent" in exc.get('name')
                and exc.get('location') is None
            ):
                match = re.findall(pattern=location_pattern, string=exc['name'])
                exc['location'] = match[0]
                if exc.get('type') == "production":
                    act['location'] = match[0]
    return db

In [37]:
# Agrifootprint
# af_path = "Data/Agrifootprint6_economic.csv"
af_path = "Data/Agrifootprint6_economic.csv"
af_name = "agrifootprint 6 economic"
af = bi.SimaProCSVImporter(
    filepath=af_path,
    name=af_name,
    delimiter=",",
)
#af = add_location_energy_acts(af)
af = add_location(af)

af.apply_strategies()
af.match_database("biosphere3", fields=("name", "unit", "categories"))
#af.match_database()
af.statistics()

Extracted 3771 unallocated datasets in 13.82 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows


(4841, 456120, 59149)

# strategies and migrations

Strategies are just functions that modify your database data. They are used when you can define rules or some kind of steps on how to modify exchange names, categories, etc so that the exchanges can be linked to existing databases. The stategies can be applied to a bunch of exchanges that need the same fix.

In [5]:
# Change names to contain "in ground"
def change_in_ground_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            if (
                exc.get('type') == 'biosphere'
                and "in ground" in exc.get('categories')
                and " " not in exc.get('name')
                and "in ground" not in exc.get('name')
            ):
                exc['name'] += ", in ground" 
    return db

In [6]:
# add "in ground" to cagegories
def change_in_ground_categories_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            if (
                exc.get('type') == 'biosphere'
                and "in ground" in exc.get('name')
                and ('natural resource', ) == exc.get('categories')
            ):
                exc['categories'] = ('natural resource', 'in ground')
    return db

In [7]:
# change names of unlinked containing "water"
def change_water_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            if exc.get('type') == 'biosphere' and "Water, " in exc.get('name'):
                x = exc['name'].split(", ")
                exc_name = f"{x[0]}, {x[1]}" 
                if "Water, cooling" in exc_name:
                    exc['name'] = 'Water, cooling, unspecified natural origin'
                    exc['categories'] = ('natural resource', 'in water')
                    if exc['unit'] == "kilogram":
                        exc['amount'] /= 1000
                        exc['unit'] = 'cubic meter'
                elif "Water, turbine use" in exc_name:
                    exc['name'] = 'Water, turbine use, unspecified natural origin'
                    exc['categories'] = ('natural resource', 'in water')
                elif "Water, river" in exc_name or "Water, lake" in exc_name:
                    exc['name'] = exc_name
                    exc['categories'] = ('natural resource', 'in water')    
                elif "Water, well" in exc_name:
                    exc['name'] = 'Water, well, in ground'
                    exc['categories'] = ('natural resource', 'in water')
                elif "Water, salt" in exc_name:
                    exc['name'] = 'Water, salt, ocean'
                    exc['categories'] = ('natural resource', 'in water')
                elif exc_name in ['Water, BR-Mid-western grid', 
                                  'Water, BR-South-eastern grid',
                                  'Water, Europe without Austria',
                                  'Water, Europe without Switzerland and Austria',
                                  'Water, RER w/o RU',
                                  'Water, unspecified natural origin',
                                  'Water, fresh']:
                    exc['name'] = 'Water, unspecified natural origin'
                    exc['categories'] = ('natural resource', 'in water')
                    if exc['unit'] == "litre":
                        exc['amount'] /= 1000
                        exc['unit'] = 'cubic meter'
    return db

In [8]:
# change names of unlinked containing "nitrogen"
def change_nitrogen_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            if exc.get('type') == 'biosphere':
                if (
                    "Nitrogen, atmospheric" in exc.get('name')
                    or "Nitrogen, total" in exc.get('name')
                ):
                    exc['name'] = "Nitrogen" 
                elif (
                    "Nitrogen dioxide" in exc.get('name')
                    and exc.get('categories') == ('water', 'ground-')
                ):
                    exc['name'] = "Nitrogen dioxide" 
                    exc['categories'] = ('water', 'surface water')
                elif (
                    "Nitrogen monoxide" in exc.get('name')
                    or "Nitrogen oxides" in exc.get('name')
                    or "Nitrogen dioxide" in exc.get('name')
                ):
                    exc['name'] = "Nitrogen oxides" 
                elif (
                    "Nitrogen, NO" in exc.get('name')
                    or "Nitrogenous Matter (unspecified, as N)" in exc.get('name')
                ):
                    exc['name'] = "Nitrogen" 
    return db

In [9]:
# Change names to contain "NMVOC"
def change_NMVOC_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            exc_name = exc.get('name')
            if (
                exc.get('type') == 'biosphere'
                and "NMVOC" in exc_name
                and ", unspecified origin" not in exc_name
            ):
                exc['name'] += ", unspecified origin" 
    return db

In [10]:
# remove locations
def change_remove_location_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            exc_name = exc.get('name')
            if (
                exc.get('type') == 'biosphere'
                and ',' in exc_name
                and ('Ammonia' in exc_name
                or 'Nitrate' in exc_name
                or 'Phosphorus' in exc_name
                or 'Sulfur dioxide' in exc_name)
            ):
                x = exc_name.split(", ")
                exc['name'] = x[0]
    return db


In [11]:
# rename PMs
def change_PM_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            exc_name = exc.get('name')
            if (
                exc.get('type') == 'biosphere'
                and exc_name in ['Particulates, < 10 um', 'Particulates, SPM', 'Particulates, unspecified']
            ):
                exc['name'] = 'Particulates, > 2.5 um, and < 10um'
    return db


In [12]:
# remove peat oxidation
def change_remove_peat_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            exc_name = exc.get('name')
            if (
                exc.get('type') == 'biosphere'
                and ', peat oxidation' in exc.get('name')
            ):
                x = exc_name.split(", ")
                exc['name'] = x[0]
    return db

In [13]:
# LUC
def change_LUC_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            exc_name = exc.get('name')
            if exc.get('type') == 'biosphere':
                if (
                    'land' not in exc.get('categories')
                    and ('Transformation,' in exc.get('name')
                    or 'Occupation,' in exc.get('name'))
                ):
                    exc['categories'] = ('natural resource', 'land')
                if (
                    'Transformation, to annual crop' in exc_name
                    or 'Transformation, to permanent crop' in exc_name
                    or 'Transformation, to grassland/pasture/meadow' in exc_name
                    or 'Transformation, from annual crop' in exc_name
                    or 'Transformation, from permanent crop' in exc_name
                    or 'Transformation, from grassland/pasture/meadow' in exc_name
                    or 'Occupation, permanent crop' in exc_name
                    or 'Occupation, annual crop' in exc_name
                    or 'Occupation, grassland/pasture/meadow' in exc_name
                ):
                    x = exc_name.split(", ")
                    exc['name'] = f"{x[0]}, {x[1]}"
                elif 'Transformation, from forest, extensive' in exc_name:
                    x = exc_name.split(", ")
                    exc['name'] = f"{x[0]}, {x[1]}, {x[2]}"
    return db


In [14]:
# energy
def change_energy_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            exc_name = exc.get('name')
            if exc.get('type') == 'biosphere':
                if (
                    'Energy, potential (in hydropower reservoir), converted' == exc_name
                    or 'Energy, from hydro power' == exc_name
                ):
                    exc['categories'] = ('natural resource', 'in water')
                    exc['name'] = 'Energy, potential (in hydropower reservoir), converted'
                elif 'Energy, from biomass' == exc_name:   
                    exc['categories'] = ('natural resource', 'biotic')
                    exc['name'] = 'Energy, gross calorific value, in biomass'
                elif 'Energy, from wood' == exc_name:   
                    exc['categories'] = ('natural resource', 'biotic')
                    exc['name'] = 'Energy, gross calorific value, in biomass, primary forest'
    return db

In [15]:
# add elements to categories
soil_agri_list = [act['name'] for act in bio if "agricultural" in act['categories']]
soil_list = [act['name'] for act in bio if ('soil', ) == act['categories']]
soil_check_list = [x for x in soil_agri_list if x not in soil_list]
water_surface_list = [act['name'] for act in bio if "surface water" in act['categories']]
water_list = [act['name'] for act in bio if ('water', ) == act['categories']]
water_check_list = [x for x in water_surface_list if x not in water_list]
air_high_list = [act['name'] for act in bio if "non-urban air or from high stacks" in act['categories']]
air_list = [act['name'] for act in bio if ('air', ) == act['categories']]
air_check_list = [x for x in air_high_list if x not in air_list]

def change_add_elements_categories_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            if exc.get('type') == 'biosphere':
                if (
                    ('soil',) == exc.get('categories')
                    and exc.get('name') in soil_check_list
                ):
                    exc['categories'] = ('soil', 'agricultural')
                elif (
                    ('water',) == exc.get('categories')
                    and exc.get('name') in water_check_list
                ):
                    exc['categories'] = ('water', 'surface water')
                elif (
                    ('air',) == exc.get('categories')
                    and exc.get('name') in air_check_list
                ):
                    exc['categories'] = ('air', 'non-urban air or from high stacks')
    return db

In [16]:
def change_categories_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            if exc.get('type') == 'biosphere':
                if (
                    'Magnesium, 0.13% in water' == exc.get('name')
                    and ('natural resource', 'in ground') == exc.get('categories')
                ):
                    exc['categories'] = ('natural resource', 'in water')
                elif (
                    'Wood, soft, standing' == exc.get('name')
                    and ('natural resource', 'in ground') == exc.get('categories')
                ):
                    exc['categories'] = ('natural resource', 'biotic')
                elif (
                    'Fish' in exc.get('name')
                    and ('natural resource', 'in water') == exc.get('categories')
                ):
                    exc['categories'] = ('natural resource', 'biotic')
                elif (
                    'Methane' == exc.get('name')
                    and ('air', ) == exc.get('categories')
                ):
                    exc['categories'] = ('air', 'urban air close to ground')
                elif (
                    'Phosphorus' == exc.get('name')
                    and ('natural resource', ) == exc.get('categories')
                ):
                    exc['categories'] = ('natural resource', 'in ground')
                    exc['name'] = 'Phosphorus, in ground'
                elif (
                    'Pyraclostrobin (prop)' == exc.get('name')
                    and 'water' in exc.get('categories')
                ):
                    exc['name'] = 'Pyraclostrobin'
                elif (
                    'Sylvite, 25 % in sylvinite, in ground' == exc.get('name')
                    and ('natural resource',) == exc.get('categories')
                ):
                    exc['categories'] = ('natural resource', 'in ground')
                elif (
                    'Hydrochloric acid' == exc.get('name')
                    and 'water' in exc.get('categories')
                ):
                    exc['categories'] = ('water', )
                elif (
                    exc.get('name') in ['Nitrate', 'Chlorine', 'PAH, polycyclic aromatic hydrocarbons',
                                       'Sulfate']
                    and 'soil' in exc.get('categories')
                ):
                    exc['categories'] = ('soil', )
                elif (
                    exc.get('name') in ['Azoxystrobin', 'Metribuzin', 'Diquat dibromide', 
                                        'Chlorpyrifos', 'Imidacloprid']
                    and 'water' in exc.get('categories')
                ):
                    exc['categories'] = ('water', 'ground-')
                    
    return db

In [17]:
# Change avoided products to inputs from technosphere
def change_substitution_acts(db):
    for act in db.data:
        for exc in act.get('exchanges'):
            if exc.get('type') == 'substitution':
                exc['type'] = 'technosphere'
                exc['amount'] *= -1
                exc['loc'] *= -1
    return db

In [18]:
# Technosphere, change units
def change_unit_acts(db):
    for act in af.data:
        for exc in act.get('exchanges'):
            if (
                "Heat" in exc.get('name')
                and "ecoinvent" in exc.get('name')
                and "kilowatt hour" == exc.get('unit')
                and exc.get('input') is None
            ):
                exc['unit'] = "megajoule"
                exc['amount'] *= 3.6
                exc['loc'] *= 3.6
            elif (
                'Wastewater, average {RoW}' in exc.get('name')
                and 'litre' == exc.get('unit')
            ):
                exc['unit'] = "cubic meter"
                exc['amount'] *= 0.001
                exc['loc'] *= 0.001
    return db

In [19]:
def allocation(db):
    for act in af.data:
        allocation = [exc.get('allocation') for exc in act.get('exchanges') if exc.get('type')=='production'][0]
        if allocation != 100 and allocation is not None:
            for exc in act.get('exchanges'):
                if exc.get('type') != 'production':
                    exc['amount'] *= allocation/100
                    exc['loc'] *= allocation/100
    return db

In [20]:
migration_name = "agrifootprint-6-names"
bi.Migration(migration_name).write(
    json.load(open("Data/agrifootprint-6-economic.json")),
    "Change names of agrifootprint activities",
)

In [38]:
af.migrate(migration_name)
af = change_in_ground_acts(af)
af = change_in_ground_categories_acts(af)
af = change_water_acts(af)
af = change_nitrogen_acts(af)
af = change_NMVOC_acts(af)
af = change_remove_location_acts(af)
af = change_PM_acts(af)
af = change_remove_peat_acts(af)
af = change_LUC_acts(af)
af = change_energy_acts(af)
af = change_categories_acts(af)
af = change_add_elements_categories_acts(af)
af = change_substitution_acts(af)
#af = add_location(af)
af = change_unit_acts(af)
af.migrate(migration_name)
#af = allocation(af)
af.apply_strategies()
af.match_database("biosphere3", fields=("name", "unit", "categories"))
af.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosp

(4841, 456120, 7642)

# Write biosphere database

Once you reach the point when you cannot link any more biosphere exchanges, and the remaining biosphere activities should just form a new database, then you need to write it as follows:

In [39]:
ag_bio_name = "agrifootprint biosphere"
try:
    del bd.databases[ag_bio_name]
except:
    pass
#af.create_new_biosphere(ag_bio_name)
bd.Database(ag_bio_name).register()
af.add_unlinked_flows_to_biosphere_database(ag_bio_name)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/13/2022 13:15:16
  Finished: 12/13/2022 13:15:16
  Total time elapsed: 00:00:00
  CPU %: 80.10
  Memory %: 6.13
Applying strategy: link_iterable_by_fields


# Write agrifootprint technosphere database

Finally, write technosphere database

In [40]:
af.drop_unlinked(i_am_reckless=True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.08 seconds


In [41]:
#af.add_unlinked_activities()
af.statistics()

4841 datasets
456051 exchanges
0 unlinked exchanges
  


(4841, 456051, 0)

In [42]:
# After you deal with the substitution exchanges, you should be able to write the database:
ag_bio_name = "agrifootprint 6 economic"
try:
    del bd.databases[ag_bio_name]
except:
    pass
af.write_database()

Writing activities to SQLite3 database:


Vacuuming database 


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Title: Writing activities to SQLite3 database:
  Started: 12/13/2022 13:15:54
  Finished: 12/13/2022 13:16:30
  Total time elapsed: 00:00:36
  CPU %: 75.40
  Memory %: 6.55
Created database: agrifootprint 6 economic


Brightway2 SQLiteBackend: agrifootprint 6 economic

# In case you want to share this bw project with others

In [26]:
bi.backup_project_directory(project)

Creating project backup archive - this could take a few minutes...


In [ ]:
# The created file is saved in your home directory. Other ppl can use it by running:
bi.restore_project_directory("give project path")
bd.projects.set_current(project)

# LCIA

In [43]:
afdb = bd.Database('agrifootprint 6 economic')

In [44]:
len(afdb)

4841

In [45]:
product_list = [act for act in afdb if 'Barley' in act['name']
                    and 'AR' in act['name']
                    and 'at farm' in act['name']
]
product_list

['Barley straw, at farm {AR} Energy, U' (kilogram, AR, None),
 'Barley grain, at farm {AR} Energy, U' (kilogram, AR, None)]

In [46]:
method_list = []
for method in bd.methods:
    if 'EF v3.0' in method and "global warming potential (GWP100)" in method:
        method_list.append(method)


In [47]:
for product in product_list:
    for lcia_method in method_list:
        lca = bc.LCA({product: 1}, method=lcia_method)
        lca.lci()
        lca.lcia()
        print(product, lcia_method[1],lca.score)

'Barley straw, at farm {AR} Energy, U' (kilogram, AR, None) climate change 3.164372971768886
'Barley straw, at farm {AR} Energy, U' (kilogram, AR, None) climate change: biogenic 0.000274864106673702
'Barley straw, at farm {AR} Energy, U' (kilogram, AR, None) climate change: fossil 0.2540552421932238
'Barley straw, at farm {AR} Energy, U' (kilogram, AR, None) climate change: land use and land use change 2.9100428654689883
'Barley grain, at farm {AR} Energy, U' (kilogram, AR, None) climate change 3.029741333491451
'Barley grain, at farm {AR} Energy, U' (kilogram, AR, None) climate change: biogenic 0.0002631697187759131
'Barley grain, at farm {AR} Energy, U' (kilogram, AR, None) climate change: fossil 0.24324618972861348
'Barley grain, at farm {AR} Energy, U' (kilogram, AR, None) climate change: land use and land use change 2.7862319740440618
